In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from moviepy.editor import VideoFileClip

In [2]:
video_file = "PATH"
audio_file = "PATH"
transcript_file = "PATH.txt"

In [3]:
# Extract audio from the video
video = VideoFileClip(video_file)
audio = video.audio
audio.write_audiofile(audio_file)

MoviePy - Writing audio in Aula_23_10_2024FRAMEWORKSPARADEEPLEARNING-pt2.wav


MoviePy - Done.


In [4]:

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]
result = pipe("PATHAUDIO", generate_kwargs={"language": "portuguese"})

with open(transcript_file, "w", encoding="utf-8") as txt_file:
    txt_file.write(result["text"])

You have passed language=portuguese, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=portuguese.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
